<a href="https://colab.research.google.com/github/mkm-world/Object-Detection-with-YOLO-workshop/blob/main/Object_Detection_with_Yolov5_Workshop_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection with YOLOv5 Workshop

This notebook contains a step by step guide of object detection with YOLOv5 on Google Colaboratory. Presented at the Datafest Africa 2024.

Check project out on GitHub [here](https://github.com/mkm-world/Object-Detection-with-YOLO-workshop)  

# Setup: Clone YOLOv5 Repository

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [1]:
# Clone the official YOLOv5 repository from GitHub
git clone https://github.com/ultralytics/yolov5  # clone

# Change the current working directory to the cloned YOLOv5 directory
%cd yolov5

# Install the required dependencies listed in the requirements.txt file, along with comet_ml for experiment tracking
%pip install -qr requirements.txt comet_ml  # install

# Import necessary libraries
import torch  # PyTorch is required for model training and inference
import utils  # Utility functions from YOLOv5 for easier workflow management
import warnings  # Handle and suppress warnings during execution

# Suppress warning messages to make the output cleaner
warnings.filterwarnings('ignore')

# Initialize the notebook environment (checks if the notebook is properly set up, including environment details)
display = utils.notebook_init()  # checks


YOLOv5 🚀 v7.0-374-g94a62456 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.2/112.6 GB disk)


# Download Data
Next, we download our dataset to the colab session. The dataset we will be working with is the Garbage Detection Dataset on [Roboflow ](https://universe.roboflow.com/thai-pham-y5sob/garbage-classify-9ndxx)

In [5]:
# Download Dataset from google drive
print("Downloading Dataset...\n")
!gdown --id 15eDCkunRYtNePfhst0KkEQubEDtPecS- -O ../garbage-detection.zip

# Unzip the dataset
print("Download Complete!")

print("\nUnzipping Files..\n")

# unzip the dataset
!unzip -q /content/garbage-detection.zip -d /content/garbage-detection-dataset

print("Unzip Complete!")


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=15eDCkunRYtNePfhst0KkEQubEDtPecS-
From (redirected): https://drive.google.com/uc?id=15eDCkunRYtNePfhst0KkEQubEDtPecS-&confirm=t&uuid=87fcb871-7f95-4fb3-a34f-3228b4558a43
To: /content/garbage-detection.zip
100% 316M/316M [00:06<00:00, 45.2MB/s]
Download Complete!

Unzipping Files..

Unzip Complete!


# Train Model

Now, we prepare to train our model. By modfiying the `data.yaml` file in our dataset's folder. Replace the content of the file with the code below.

Lastly, we specify our model's training parameters and paths as seen below.

Modify the config file
```yaml
# Class names for the object detection task
# Each number corresponds to a class label for different types of garbage
names:
  0: can     # Class 0 represents cans (e.g., soda cans, tin cans)
  1: plastic # Class 1 represents plastic items (e.g., bottles, bags)
  2: glass   # Class 2 represents glass items (e.g., glass bottles, jars)
  3: paper   # Class 3 represents paper items (e.g., cardboard, paper sheets)

# Number of classes
nc: 4  # We have 4 classes in total: can, plastic, glass, and paper

# Paths to training and validation image datasets
train: /content/garbage-detection-dataset/garbage-detection-dataset/train/images
# Specifies the path to the folder containing training images

val: /content/garbage-detection-dataset/garbage-detection-dataset/valid/images
# Specifies the path to the folder containing validation images

```






In [9]:
# Train YOLOv5s on our dataset for 30 epochs
!python train.py --img 640 --batch 16 --epochs 30 --data "/content/garbage-detection-dataset/garbage-detection-dataset/data.yaml" --weights yolov5s.pt --cache --exist-ok

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      15/29      4.59G    0.02838    0.01604    0.01061         46        640:  46%|████▋     | 76/164 [00:26<00:26,  3.33it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      15/29      4.59G    0.02837    0.01605    0.01056         48        640:  47%|████▋     | 77/164 [00:26<00:24,  3.49it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      15/29      4.59G    0.02844    0.01604    0.01052         40        640:  48%|████▊     | 78/164 [00:26<00:24,  3.57it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead

# View Model Results

## Download yolo results

In [13]:
# Get the current directory
current_dir = '/content/yolov5/runs/train/exp'

import os
import zipfile
from google.colab import files

# Create a list of all files and directories in the current directory
file_list = os.listdir(current_dir)

# Filter out directories (if any)
file_list = [f for f in file_list if os.path.isfile(os.path.join(current_dir, f))]

# Create a zip file
zip_filename = "files.zip"
with zipfile.ZipFile(zip_filename, "w") as zip_file:
    for file in file_list:
        file_path = os.path.join(current_dir, file)
        zip_file.write(file_path)

# Download the zip file
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test Model in real-life

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.25 --exist-ok --source '/content/demo-video.mp4'
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/VID_20240614_151537.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-353-g5eca7b9c Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5a9c782b3dc0] Detected moov in a free or hoov atom.
video 1/1 (1/316) /content/VID_20240614_151537.mp4: 640x384 (no detections), 65.4ms
video 1/1 (2/316) /content/VID_20240614_151537.mp4: 640x384 (no detections), 8.9ms
video 1/1 (3/316) /content/VID_20240614_151537.mp4: 64

# What's next?

### Is performance Satisfactory? If No,
- Re-training with more data
- Augmentation?
- Experimenting with different set of hyperparameters.

### If satisfactory,
Export trained model, can be deployed on the web, locally, on mircrocontrollers like the raspberry pi, NIVIDIA Jetson Nano,etc. Some of the popular export formats are listed below.
- `.pt` -  pytorch model(default)
- `.tflite` tensorflow lite model for deployment on mobile and low-end devices.
- `.onnx`- popularly used for low end devices.

The object detection model has different applications:
- Object counting
- Defect Monitoring production lines.
- Object Blurring.
- Line Passage Counter
- First stage for Image recogniton algorithm, e.g Number Plate Detector.